# Sap flow prediction Ananlysis

# Introduction
This notebook walks through an analysis of sap flows from maple trees in the US northeast and in Quebec (Canada).  Specifically, the methodology proposed by Houle et al. (2015) is applied to sap flow data made published by Stinson et al. (2019).  The analysis involves predicting whether or not there will be sap collected during a given week using a logistic regression model based on the derived weather parameters Growing Degree Days (GDD) and freeze-thaw cycles (frthw).  A comparison is made between the precision achieved by Houle et al. (2015) on their original dataset and the current data set using the same modelling parameters.

In [2]:
# Import required pacakages
import numpy as np
import pandas as pd
import os
import altair as alt
from vega_datasets import data

# Handle large data sets without embedding them in the notebook
alt.data_transformers.enable("data_server")

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Data Preparation
The following code chunk loads the tables which have been created from the data set provided by Stinson et al. (2019) on [ScienceBase](https://www.sciencebase.gov/catalog/item/5d67eacae4b0c4f70cf15be3).  The scripts used to create these tables can be found in the [src](https://github.com/spentelow/sapflow/tree/main/src).

In [3]:
# Load sap measurement data
norm_path = os.path.join(os.pardir, "data", "processed", "stinson2019", "norm_tables")
derived_path = os.path.join(os.pardir, "data", "processed", "stinson2019", "derived_tables")
sap_sugar_df = pd.read_pickle(os.path.join(derived_path, 'sap_sugar_weekly_summary'))

# Load freeze/thaw and growing degree data for a given weather station and associate with a sap measurement site
gdd_frthw = pd.read_pickle(os.path.join(derived_path,'gdd_frthw'))
closest_weather_stn = pd.read_pickle(os.path.join(norm_path, 'closest_weather_stn'))
gdd_frthw = gdd_frthw.merge(closest_weather_stn.reset_index(), how = 'left', on= 'stn_id')
gdd_frthw = gdd_frthw.set_index('datetime')

# Load information on sap measurement sites
location = pd.read_pickle(os.path.join(norm_path, 'location'))
weather_stn = pd.read_pickle(os.path.join(norm_path, 'weather_stn'))

## Locations

The following plot shows the location of the study site for the sap data collection along with the NOAA weather station which has been associated with each site for the purposes of this analysis

In [21]:
# **TODO**
# Get map showing outlines of Canadian Provinces and US States.
# **TODO**

site_points = alt.Chart(location).mark_circle().encode(
    longitude='lon:Q',
    latitude='lat:Q',
    size=alt.value(30)
)

site_labels_east = alt.Chart(location[location.lon>-80]).mark_text(
    align='left',
    baseline='middle',
    dx=5,
    dy=-3
).encode(
    longitude='lon:Q',
    latitude='lat:Q',
    text='long_name',
    size= alt.value(10)
)

site_labels_west = alt.Chart(location[location.lon<=-80]).mark_text(
    align='right',
    baseline='middle',
    dx=-5,
    dy=-3
).encode(
    longitude='lon:Q',
    latitude='lat:Q',
    text='long_name',
    size= alt.value(10)
)

site_plt = site_points + site_labels_east + site_labels_west

# Weather station points and text

stn_points = alt.Chart(weather_stn).mark_circle(color='black').encode(
    longitude='lon:Q',
    latitude='lat:Q',
    size=alt.value(10)
)

stn_labels_east = alt.Chart(weather_stn[weather_stn.lon>-80]).mark_text(
    align='left',
    baseline='middle',
    dx=5,
    dy=+7,
    color='grey'
).encode(
    longitude='lon:Q',
    latitude='lat:Q',
    text='stn_name',
    size= alt.value(8)
)

stn_labels_west = alt.Chart(weather_stn[weather_stn.lon<=-80]).mark_text(
    align='right',
    baseline='middle',
    dx=-5,
    dy=+7,
    color='grey'
).encode(
    longitude='lon:Q',
    latitude='lat:Q',
    text='stn_name',
    size= alt.value(8)
)

stn_plt = stn_points + stn_labels_east + stn_labels_west

# states = alt.topo_feature(data.us_10m.url, feature='states')
states = alt.topo_feature(data.world_110m.url, 'countries')

background = alt.Chart(states).mark_geoshape(
    fill='lightgray',
    stroke='white'
)

(background + site_plt + stn_plt).project('equirectangular',
                                        scale=1200,
                                        translate=[2200, 1100]
                                       ).properties(width=1000,height=600)


alt.LayerChart(...)

## Analysis Table Creation

The following code chunk creates a dataframe which draws together information on the parameters required for the prediction model used by Houle et al. (2015).  These parameters are:
* Cumulative Growing Degree Days (GDD)
* Cumulative freeze-thaw cycles (frthw)
* Whether sap flow occurred during a given week sap flow (0 = no, 1 = yes)

Additional parameters are also given in the dataframe `full_sap` including the measurement dates and locations.

In [46]:
# Create a dataframe (`full_sap`) which contains information on both sap flow and the growing degree days and freeze/thaw cycles for all tap/year combinations

full_sap = pd.DataFrame(
    columns=[
        "tap_id",
        "date_from",
        "date_to",
        "weekly_sugarwt",
        "weekly_sap",
        "site",
        "cumGDD",
        "cum_frthw",
        "sap_binary",
    ]
)

for site in sap_sugar_df.site.unique():
    sap_sugar_site = sap_sugar_df[sap_sugar_df.site == site]
    for tap in sap_sugar_site.tap_id.unique():
        sap_sugar_tap = sap_sugar_site[sap_sugar_site.tap_id == tap]
        for year in sap_sugar_tap.date_to.dt.year.unique():
            sap_sugar_year = sap_sugar_tap[sap_sugar_tap.date_to.dt.year == year].drop(
                columns=["site"]
            )

            # Merge weather from local station with sap measurements for a given tap and year
            sap_sugar_year = sap_sugar_year.merge(
                gdd_frthw[(gdd_frthw.site == site) & (gdd_frthw.index.year == year)],
                how="right",
                left_on=["date_to"],
                right_index=True,
            )

            # Add missing dates, tapids to df
            sap_sugar_year.loc[:, "date_from"] = sap_sugar_year.loc[
                :, "date_to"
            ] - pd.DateOffset(6)
            sap_sugar_year["weather_datetime"] = sap_sugar_year.date_to
            sap_sugar_year["tap_id"] = sap_sugar_year["tap_id"].fillna(tap)

            # Fill in missing 0 values
            sap_sugar_year.loc[:, "weekly_sugarwt"] = sap_sugar_year.loc[
                :, "weekly_sugarwt"
            ].fillna(0)
            sap_sugar_year.loc[:, "weekly_sap"] = sap_sugar_year.loc[
                :, "weekly_sap"
            ].fillna(0)

            #             # For future implementation
            #             # Add total weekly freeze-thaw cycles column (week ending on date 'date_to')
            #             sap_sugar_year.loc[:, "weekly_frthw"] = sap_sugar_year.loc[:, "cum_frthw"] - sap_sugar_year.loc[:, "cum_frthw"].shift(6)
            sap_sugar_year = sap_sugar_year.rename(columns={'frthw':'cum_frthw'})
            

            # Create row indicating if there is (1) or is not (0) sap flow in a given week (week ending on 'date_to')
            sap_sugar_year["sap_binary"] = sap_sugar_year["weekly_sap"].apply(
                lambda x: 0 if ((pd.isnull(x)) | (x == 0)) else 1
            )

            full_sap = full_sap.append(sap_sugar_year)

# Coerce sap_binary column to numpy.int64 for subesquent analyses
full_sap.sap_binary = full_sap.sap_binary.astype("int")

# Drop columns not used in current analysis and reset index
full_sap = full_sap.drop(columns=['weekly_sugarwt', 'weekly_sap','mean_airt'])
full_sap.reset_index(inplace=True, drop=True)

In [42]:
full_sap.head()

,tap_id,date_from,date_to,site,cumGDD,cum_frthw,sap_binary,stn_id,GDD,weather_datetime
0,DOF1A,2013-12-26,2014-01-01,DOF,0.0,0.0,0,726116-94765,0.0,2014-01-01
1,DOF1A,2013-12-27,2014-01-02,DOF,0.0,0.0,0,726116-94765,0.0,2014-01-02
2,DOF1A,2013-12-28,2014-01-03,DOF,0.0,0.0,0,726116-94765,0.0,2014-01-03
3,DOF1A,2013-12-29,2014-01-04,DOF,0.0,0.0,0,726116-94765,0.0,2014-01-04
4,DOF1A,2013-12-30,2014-01-05,DOF,0.0,0.0,0,726116-94765,0.0,2014-01-05


# Analysis

## Logistic Regression Model

Houle et al. (2015) created a logistic regression model to predict the presence or absence of maple syrup production in a given week.  The following linear function was developed based on their measured data:

$$ P = -5.09 + 0.722F - 0.014F^2 - 0.07G$$

Where:

$P$ = Predictor of whether there will or will not be sap flowing in a given week (variable is labelled 'Production' in Houle et al., 2015)

$F$ = Cumulative number of freeze/thaw events since the beginning of the year (January 1st).  A freeze/thaw event is counted if the temperature rises above a given threshold ($T_{thresh}$) and drops below it again.  A threshold of 3&deg;C has been used as in Houle et al., 2015.

$G$ = Cumulative number of growing degree days since the beginning of the year (January 1st) using a 5&deg;C base temperature ($T_{base}$).  Each day, the maximum air temperature ($T_{max}$is extracted and, if it is above the $T_{base}$, a value of $T_{base} - T_{max}$ is added to the running total of growing degree days ($G$).

Passing $P$ into a sigmoid function and applying a threshold of 0.51, we end up with a prediction of whether there will or will not be sap flow in a given week.

$$\hat{Y} = \begin{cases}
1 & \text{if} \ \frac{1}{1+e^{-P}} \geq 0.51 \\
0 & \text{if} \ \frac{1}{1+e^{-P}} < 0.51
\end{cases}
$$

*Note that in the subsequent tables, $S$ is used to denote the output of the sigmoid function $\frac{1}{1+e^{-P}}$.*

The code chunks below manually compute the predictions of for $P$ for the data provided by Stinson et al. (2019) and stores the full result in a dataframe called `LR_table`.  A summary of the prediction results for each tap and year are stored in `LR_summary` including whether a prediction was a true positive, true negative, false positive, or false negative and the precision of the predictions with respect to weeks when sap flow did occur (`precision_1`) and for weeks when sap flow did not occur (`precision_0`). 

In [54]:
# Coefficients of linear model created by Houle 2015 and threshold for predicting 'True' for sap flow from logistic function
houle_coeff = np.array([-5.09, 0.733, -0.014, -0.07])
houle_thresh = 0.51

# Create dataframe of parameters required for Houle 2015 analysis
LR_table = full_sap[
    ["site", "tap_id", "date_from", "date_to", "cum_frthw", "cumGDD", "sap_binary"]
]
LR_table = LR_table.rename(
    columns={"cum_frthw": "F", "cumGDD": "G", "sap_binary": "Y"}
)

LR_table["F2"] = LR_table.F ** 2  # Add F**2 column
LR_table["bias"] = 1  # Add bias column


# Calculations to generate weekly sapflow predictions based on model by Houle et al., 2015
LR_table.loc[:, "P"] = LR_table[["bias", "F", "F2", "G"]] @ (houle_coeff)
LR_table.loc[:, "S"] = 1 / (1 + np.exp(-LR_table["P"])
)  # Add intermediate sigmoid output column 'S'
LR_table.loc[:, "Y_hat"] = (LR_table["S"] > houle_thresh).astype(
    "int"
)  # Threshold determined by Houle 2015
LR_table.loc[:, "correct"] = (LR_table.Y_hat == LR_table.Y).astype(
    "int"
)  # Add column indicating if prediction was correct or not


LR_table["jd"] = LR_table["date_to"].dt.dayofyear
LR_table["year"] = LR_table["date_to"].dt.year

# Add columns to count number of true positives, false positives, false negatives, and true negatives
LR_table["tp"] = LR_table.apply(
    lambda x: 1 if (x.Y == 1 and x.Y_hat == 1) else 0, axis=1
)
LR_table["fp"] = LR_table.apply(
    lambda x: 1 if (x.Y == 0 and x.Y_hat == 1) else 0, axis=1
)
LR_table["fn"] = LR_table.apply(
    lambda x: 1 if (x.Y == 1 and x.Y_hat == 0) else 0, axis=1
)
LR_table["tn"] = LR_table.apply(
    lambda x: 1 if (x.Y == 0 and x.Y_hat == 0) else 0, axis=1
)

In [55]:
LR_table.head()

,site,tap_id,date_from,date_to,F,G,Y,F2,bias,P,S,Y_hat,correct,jd,year,tp,fp,fn,tn
0,DOF,DOF1A,2013-12-26,2014-01-01,0.0,0.0,0,0.0,1,-5.09,0.00612,0,1,1,2014,0,0,0,1
1,DOF,DOF1A,2013-12-27,2014-01-02,0.0,0.0,0,0.0,1,-5.09,0.00612,0,1,2,2014,0,0,0,1
2,DOF,DOF1A,2013-12-28,2014-01-03,0.0,0.0,0,0.0,1,-5.09,0.00612,0,1,3,2014,0,0,0,1
3,DOF,DOF1A,2013-12-29,2014-01-04,0.0,0.0,0,0.0,1,-5.09,0.00612,0,1,4,2014,0,0,0,1
4,DOF,DOF1A,2013-12-30,2014-01-05,0.0,0.0,0,0.0,1,-5.09,0.00612,0,1,5,2014,0,0,0,1


In [125]:
# Calculate summary of performance of Houle et al., 2015 model in terms of precision of prediction for weeks with sap flow (`precision_1`) 
# and prediction of weeks without sap flow (`precision_0`).
LR_summary = (
    LR_table[["site", "tap_id", "year", "tn", "fp", "fn", "tp"]]
    .groupby(["tap_id", "year", "site"])
    .sum()
)
LR_summary["precision_1"] = LR_summary.tp / (LR_summary.tp + LR_summary.fn)
LR_summary["precision_0"] = LR_summary.tn / (LR_summary.tn + LR_summary.fp)
LR_summary = LR_summary.reset_index()
LR_summary = LR_summary.merge(
    location.reset_index()[["site", "short_name", "state_province"]],
    on="site",
    how="left",
)
LR_summary["loc"] = LR_summary.short_name + ", " + LR_summary.state_province

In [126]:
LR_summary.head()

,tap_id,year,site,tn,fp,fn,tp,precision_1,precision_0,short_name,state_province,loc
0,DOF10A,2014,DOF,265,62,21,17,0.447368,0.810398,Dartmouth,NH,"Dartmouth, NH"
1,DOF10A,2015,DOF,322,3,4,36,0.900000,0.990769,Dartmouth,NH,"Dartmouth, NH"
2,DOF10A,2016,DOF,290,28,32,16,0.333333,0.911950,Dartmouth,NH,"Dartmouth, NH"
3,DOF10A,2017,DOF,290,40,14,21,0.600000,0.878788,Dartmouth,NH,"Dartmouth, NH"
4,DOF10B,2014,DOF,265,68,21,11,0.343750,0.795796,Dartmouth,NH,"Dartmouth, NH"


# Visualization of results

## Logistic Regression Predictions

The following plot illustrates the predictions of the logistic regression model for a single tap ('QC1A' from the Boris/Quebuec site) for a single year (2015).  The plot displays the output of the sigmoid function ($S$) for the values of the linear regression ($P$)  The threshold $S$ value to predict sap in a given week was 0.51 as specified by Houle et al (2015) and is plotted as a dashed black line.

In [127]:
# Creates sample plot demonstrating output of sigmoid function over the course of a year.  Correct predictions in orange, incorrect in blue

sample_sigmoid = LR_table[
    (LR_table.site == "QC")
    & (LR_table.date_to.dt.year == 2015)
    & (LR_table.tap_id == "QC1A")
]

sample_sigmoid = sample_sigmoid.rename(
    columns={
        "tn": "No sap observed; no sap predicted by LR (tn)",
        "fp": "No sap observed; sap predicted by LR (fp)",
        "fn": "Sap observed; no sap predicted by LR (fn)",
        "tp": "Sap observed; sap predicted by LR (tp)",
    }
)

sample_sigmoid = (
    sample_sigmoid.reset_index()
    .melt(
        id_vars="index",
        value_vars=[
            "Sap observed; sap predicted by LR (tp)",
            "Sap observed; no sap predicted by LR (fn)",
            "No sap observed; no sap predicted by LR (tn)",
            "No sap observed; sap predicted by LR (fp)",
        ],
    )
    .merge(
        sample_sigmoid.reset_index()[["index", "S", "P", "Y", "Y_hat", "jd"]],
        on="index",
    )
)

sample_sigmoid = sample_sigmoid[sample_sigmoid.value == 1]


# sample_sigmoid['prediction'] = sample_sigmoid.apply(
#     lambda x: 'Correct' if (x.correct == 1) else 'Incorrect', axis=1
# )
# alt.Chart(sample_sigmoid).mark_circle().encode(
#     alt.X("jd", title="Day of year"),
#     alt.Y("S", title="Sigmoid function output"),
#     fill=alt.Fill("prediction", title="Prediction", scale=alt.Scale(
#             domain=['Correct', 'Incorrect'],
#             range=['black', 'red'])),
# )

domain_pred = [
    "Sap observed; sap predicted by LR (tp)",
    "Sap observed; no sap predicted by LR (fn)",
    "No sap observed; no sap predicted by LR (tn)",
    "No sap observed; sap predicted by LR (fp)",
]

sigmoid_plt = (
    alt.Chart(sample_sigmoid)
    .mark_point(size=100, stroke=None, opacity=0.8)
    .encode(
        alt.X("jd", title="Day of year", scale=alt.Scale(domain=[60, 150])),
        alt.Y("S", title="Sigmoid function output"),
        fill=alt.Fill(
            "variable",
            title="LR Result",
            scale=alt.Scale(domain=domain_pred, range=["green", "red", "green", "red"]),
        ),
        shape=alt.Shape(
            "variable",
            scale=alt.Scale(
                domain=domain_pred, range=["cross", "cross", "circle", "circle"]
            ),
        ),
    )
    .interactive()
)


sigmoid_plt = sigmoid_plt + alt.Chart(pd.DataFrame({"ht": [houle_thresh]})).mark_rule(
    color="black",
    strokeDash=[10,10]
).encode(y="ht")

sigmoid_plt.properties(width=800, height=500).configure_legend(labelLimit=0)

alt.LayerChart(...)

## Precision of Predictions
The following plot shows the precision of the predictions generated with the model proposed by Houle et al. (2015) for each tap and each year, .  Houle et al (2015) stated that, 'The global model accurately predicted 83% of the production weeks and 95% of the non-production weeks.'  Assuming that these values represent the precision with respect to production weeks and with respect to non-production weeks, respectively, these values have been plotted as dashed red lines for reference.

In [128]:
# Generate plots showing prediction precision by site (red lines for comparison to precision reported by Houle et al., 2015)


prec_1_plt = (
    alt.Chart(LR_summary)
    .mark_boxplot()
    .encode(
        y=alt.Y("precision_1", title="Precision of prediction of weeks with sap flow"),
        x=alt.X("loc", title="Site Location"),
        #         color = alt.Color("loc", legend=None)
    )
)

prec_1_plt = prec_1_plt + alt.Chart(houle_means).mark_rule(
    color="red", strokeDash=[10, 10]
).encode(y="prec_1")

prec_0_plt = (
    alt.Chart(LR_summary)
    .mark_boxplot()
    .encode(
        y=alt.Y(
            "precision_0", title="Precision of prediction of weeks without sap flow"
        ),
        x=alt.X("loc", title="Site Location"),
        #         color = alt.Color("loc", legend=None)
    )
)

houle_means = pd.DataFrame({"prec_1": [0.83], "prec_0": [0.95]})

prec_0_plt = prec_0_plt + alt.Chart(houle_means).mark_rule(
    color="red", strokeDash=[10, 10]
).encode(y="prec_0")


width = 500

In [129]:
prec_1_plt.properties(width=width)

alt.LayerChart(...)

In [114]:
prec_0_plt.properties(width=width)

alt.LayerChart(...)

## Site Specific Logistic Regression
Using the same model set up as Houle et al. (2015), separate logistic regression models were fit for each Site to determine the improvement in the model predictions when the model was fit to local data.  The results are plotted 

In [134]:
log_regs = dict()
for site in LR_table.site.unique():
    X = LR_table[LR_table.site==site][["F", "F2", "G"]]
    y = LR_table[LR_table.site==site][['Y']]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state = 123, stratify = LR_table[LR_table.site==site].tap_id)
    log_regs[site] = LogisticRegression()
    log_regs[site].fit(X_train, y_train)
    log_regs[site].score(X_train,y_train)



/home/steffen/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/steffen/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/steffen/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/steffen/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of

In [135]:
for site in LR_table.site.unique():
    LR_table.loc[LR_table.site==site,'Y_logreg'] = log_regs[site].predict(LR_table.loc[LR_table.site==site, ["F", "F2", "G"]])

In [136]:
LR_table.loc[:, "log_reg_correct"] = (LR_table.Y_hat == LR_table.Y_logreg).astype(
    "int"
)  # Add column indicating if prediction was correct or not

# Add columns to count number of true positives, false positives, false negatives, and true negatives
LR_table["tp_lr"] = LR_table.apply(
    lambda x: 1 if (x.Y == 1 and x.Y_logreg == 1) else 0, axis=1
)
LR_table["fp_lr"] = LR_table.apply(
    lambda x: 1 if (x.Y == 0 and x.Y_logreg == 1) else 0, axis=1
)
LR_table["fn_lr"] = LR_table.apply(
    lambda x: 1 if (x.Y == 1 and x.Y_logreg == 0) else 0, axis=1
)
LR_table["tn_lr"] = LR_table.apply(
    lambda x: 1 if (x.Y == 0 and x.Y_logreg == 0) else 0, axis=1
)

In [137]:
# Calculate summary of performance of Houle et al., 2015 model in terms of precision of prediction for weeks with sap flow (`precision_1`) 
# and prediction of weeks without sap flow (`precision_0`).
LR_summary.loc[:,["tn_lr", "fp_lr", "fn_lr", "tp_lr"]] = (
    LR_table[["site", "tap_id", "year", "tn_lr", "fp_lr", "fn_lr", "tp_lr"]]
    .groupby(["tap_id", "year", "site"])
    .sum()
).reset_index()
LR_summary["precision_1_lr"] = LR_summary.tp_lr / (LR_summary.tp_lr + LR_summary.fn_lr)
LR_summary["precision_0_lr"] = LR_summary.tn_lr / (LR_summary.tn_lr + LR_summary.fp_lr)
LR_summary

,tap_id,year,site,tn,fp,fn,tp,precision_1,precision_0,short_name,state_province,loc,tn_lr,fp_lr,fn_lr,tp_lr,precision_1_lr,precision_0_lr
0,DOF10A,2014,DOF,265,62,21,17,0.447368,0.810398,Dartmouth,NH,"Dartmouth, NH",321,6,5,33,0.868421,0.981651
1,DOF10A,2015,DOF,322,3,4,36,0.900000,0.990769,Dartmouth,NH,"Dartmouth, NH",325,0,28,12,0.300000,1.000000
2,DOF10A,2016,DOF,290,28,32,16,0.333333,0.911950,Dartmouth,NH,"Dartmouth, NH",317,1,11,37,0.770833,0.996855
3,DOF10A,2017,DOF,290,40,14,21,0.600000,0.878788,Dartmouth,NH,"Dartmouth, NH",314,16,0,35,1.000000,0.951515
4,DOF10B,2014,DOF,265,68,21,11,0.343750,0.795796,Dartmouth,NH,"Dartmouth, NH",326,7,0,32,1.000000,0.978979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
547,SMAS6A,2016,SMM,305,43,0,18,1.000000,0.876437,Southernmost,VA,"Southernmost, VA",340,8,0,18,1.000000,0.977011
548,SMAS7A,2016,SMM,305,44,0,17,1.000000,0.873926,Southernmost,VA,"Southernmost, VA",340,9,0,17,1.000000,0.974212
549,SMAS7B,2016,SMM,305,44,0,17,1.000000,0.873926,Southernmost,VA,"Southernmost, VA",340,9,0,17,1.000000,0.974212
550,SMAS8A,2016,SMM,305,43,0,18,1.000000,0.876437,Southernmost,VA,"Southernmost, VA",340,8,0,18,1.000000,0.977011


In [16]:
# Creates sample plot demonstrating output of sigmoid function over the course of a year.  Correct predictions in orange, incorrect in blue

sample_sigmoid = LR_table[
    (LR_table.site == "QC")
    & (LR_table.date_to.dt.year == 2015)
    & (LR_table.tap_id == "QC1A")
]
sample_sigmoid['prediction'] = sample_sigmoid.apply(
    lambda x: 'Correct' if (x.correct == 1 and x.log_reg_correct==1) else ('LR_Correct' if (x.correct == 0 and x.log_reg_correct==1) else ('Houle_Correct' if (x.correct == 1 and x.log_reg_correct==0) else 'Incorrect')), axis=1
)

alt.Chart(sample_sigmoid).mark_circle().encode(
    alt.X("jd", title="Day of year"),
    alt.Y("S", title="Sigmoid function output"),
    fill=alt.Fill("prediction", title="Prediction", scale=alt.Scale(
            domain=['Correct','Houle_Correct', 'LR_Correct', 'Incorrect'],
            range=['green', 'orange', 'black', 'red'])),
)

<ipython-input-16-96e64de362bc>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_sigmoid['prediction'] = sample_sigmoid.apply(


alt.Chart(...)

In [138]:
LR_prec = (
    LR_summary.reset_index()
    .melt(
        id_vars="index",
        value_vars=["precision_1", "precision_1_lr", "precision_0", "precision_0_lr"],
        value_name="prec_model",
    )
    .merge(LR_summary.reset_index(), on="index")
)

In [18]:
LR_prec.loc[(LR_prec['variable']=='precision_1') | (LR_prec['variable']=='precision_1_lr'), 'houle'] = 0.83
LR_prec

,index,variable,prec_model,tap_id,year,site,tn,fp,fn,tp,...,long_name,state_province,loc,tn_lr,fp_lr,fn_lr,tp_lr,precision_1_lr,precision_0_lr,houle
0,0,precision_1,0.447368,DOF10A,2014,DOF,265,62,21,17,...,Dartmouth Organic Farm,NH,"Dartmouth Organic Farm, NH",321,6,5,33,0.868421,0.981651,0.83
1,0,precision_1_lr,0.868421,DOF10A,2014,DOF,265,62,21,17,...,Dartmouth Organic Farm,NH,"Dartmouth Organic Farm, NH",321,6,5,33,0.868421,0.981651,0.83
2,0,precision_0,0.810398,DOF10A,2014,DOF,265,62,21,17,...,Dartmouth Organic Farm,NH,"Dartmouth Organic Farm, NH",321,6,5,33,0.868421,0.981651,NaN
3,0,precision_0_lr,0.981651,DOF10A,2014,DOF,265,62,21,17,...,Dartmouth Organic Farm,NH,"Dartmouth Organic Farm, NH",321,6,5,33,0.868421,0.981651,NaN
4,1,precision_1,0.900000,DOF10A,2015,DOF,322,3,4,36,...,Dartmouth Organic Farm,NH,"Dartmouth Organic Farm, NH",325,0,28,12,0.300000,1.000000,0.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2203,550,precision_0_lr,0.977011,SMAS8A,2016,SMM,305,43,0,18,...,Southernmost Maple,VA,"Southernmost Maple, VA",340,8,0,18,1.000000,0.977011,NaN
2204,551,precision_1,1.000000,SMAS8B,2016,SMM,305,43,0,18,...,Southernmost Maple,VA,"Southernmost Maple, VA",340,8,0,18,1.000000,0.977011,0.83
2205,551,precision_1_lr,1.000000,SMAS8B,2016,SMM,305,43,0,18,...,Southernmost Maple,VA,"Southernmost Maple, VA",340,8,0,18,1.000000,0.977011,0.83
2206,551,precision_0,0.876437,SMAS8B,2016,SMM,305,43,0,18,...,Southernmost Maple,VA,"Southernmost Maple, VA",340,8,0,18,1.000000,0.977011,NaN


In [142]:
# Generate plots showing prediction precision by site (red lines for comparison to precision reported by Houle et al., 2015

LR_prec = (
    LR_summary.reset_index()
    .melt(
        id_vars="index",
        value_vars=["precision_1", "precision_1_lr", "precision_0", "precision_0_lr"],
        value_name="prec_model",
    )
    .merge(LR_summary.reset_index(), on="index")
)

LR_prec.loc[(LR_prec['variable']=='precision_1') | (LR_prec['variable']=='precision_1_lr'), 'houle'] = 0.83
LR_prec.loc[(LR_prec['variable']=='precision_0') | (LR_prec['variable']=='precision_0_lr'), 'houle'] = 0.95

prec_1_plt_lr = (
    alt.Chart(LR_prec[(LR_prec['variable']=='precision_1') | (LR_prec['variable']=='precision_1_lr')])
    .mark_rule(color = 'red'
    ).encode(
        y='houle'
    ).mark_boxplot()
    .encode(
        y=alt.Y("prec_model", title="Precision of prediction of weeks with sap flow"),
        x=alt.X("variable", title="Site Location"),
        color = alt.Color("variable"),
        column = 'site'
    )
)

prec_0_plt_lr = (
    alt.Chart(LR_prec[(LR_prec['variable']=='precision_0') | (LR_prec['variable']=='precision_0_lr')])
    .mark_boxplot()
    .encode(
        y=alt.Y("prec_model", title="Precision of prediction of weeks with sap flow"),
        x=alt.X("variable", title="Site Location"),
        color = alt.Color("variable"),
        column = 'site'
    )
)


width = 100

ValueError: Faceted charts cannot be layered.

In [140]:
prec_1_plt_lr.properties(width=width)

alt.Chart(...)

In [141]:
prec_0_plt_lr.properties(width=width)

alt.Chart(...)